## File Linker

The goal here is to produce a map of IDs to datasets

I'll be using the two Bloomberg Estimate Americas datasets

#### Useful info

* Primary keys for BEst Americas - idBbGlobal,idBbUnique,bestFperiodOverride

* Their old mnemonic forms are ID_BB_GLOBAL, ID_BB_UNIQUE, BEST_FPERIOD_OVERRIDE

* The first three columns of a given record in an out file correspond to: record identifier, return code (meaningful in .dif files), number of columns (every record should have the same number)

The .OUT files are organized by holding the column information like this:

START-OF-FIELDS

column names

END-OF-FIELDS

and the data like this:

START-OF-DATA

data

END-OF-DATA

#### General problem setup

I want to
    * open a file, read its contents
    * separate the header info and the data
    * find my keys in the header
    * roll through the data, grabbing each record with a valid primary key
    * repeat for each dataset we're opening
    * create a map from primary key to dataset

I broke this out into two different functions since the first one was getting kinda long!

In [ ]:
def collect_keys(datasets, primary_key):
    """ return all the primary keys in some datasets
        Arguments:
            datasets (list): a list of paths to datasets
            primary_key (list): a list of fields, in their old mnemonic form
        Returns:
            a list of lists, where each 
    """
    all_keys = []

    for dataset in datasets:
        dataset_keys = []
        # open the file, read in the data
        file = open(dataset)
        file_content = file.readlines()
        clean_file_content = []
        for line in file_content:
            line = line.strip() # removes the trailing \n from each line
            clean_file_content.append(line)

        # header parsing
        start_of_header = clean_file_content.index('START-OF-FIELDS')
        end_of_header = clean_file_content.index('END-OF-FIELDS')
        header = clean_file_content[start_of_header+1:end_of_header]

        # data parsing
        start_of_data = clean_file_content.index('START-OF-DATA')
        end_of_data = clean_file_content.index('END-OF-DATA')
        data = clean_file_content[start_of_data+1:end_of_data]

        # primary key indices
        key_indices = []
        for key in primary_key:
            key_indices.append(header.index(key))
        
        # walk through data, collecting primary keys
        for row in data:
            pkey = []
            row_data = row.split('|')
            for key_index in key_indices:
                pkey.append(row_data[key_index + 3]) # offset of 3 since those cols aren't helpful here
            pkey_str = '_'.join(pkey)# produces this: BBG000BCSCB1_EQ0010017100001000_2005Q4
            dataset_keys.append(pkey_str)

        # we've collected every record in the file, time to add it to our output
        all_keys.append(dataset_keys)

    return all_keys

In [ ]:
def map_keys(datasets, all_keys):
    """ create a map of keys and datasets
        Arguments:
            datasets (list): a list of datasets
            all_keys (list): a list of lists primary keys found in each dataset
        Returns:
            a dictionary mapping the keys to datasets
    """
    # we can work this way because we know that we have the same number of datasets as key lists
    key_dict = {}
    for i in range(len(datasets)):
        for key in all_keys[i]:
            if key in key_dict:
                key_dict[key].append(datasets[i])
            else:
                key_dict[key] = [datasets[i]]
    return key_dict

In [ ]:
primary_keys = ['ID_BB_GLOBAL', 'ID_BB_UNIQUE', 'BEST_FPERIOD_OVERRIDE']
datasets = ['data/20191212_best_amer.out', 'data/20191204_best_amer.out']
key_lists = collect_keys(datasets, primary_keys)
key_dict = map_keys(datasets, key_lists)

with open('example_key_mapping.csv', 'w') as output_file:
    # remember, dictionary.items() gives us the key, value pairs in our for loop!
    for key, file_list in key_dict.items():
        file_str = ','.join(file_list)
        output_file.write(key + ',' + file_str + '\n')

## .OUT file parser

Shares a bit of similarities with the first function for file linking


#### General problem setup
    * Open the file, read in the data
    * separate the header info and the data
    * load into pandas
    * save to csv

In [ ]:
import pandas as pd

def out_to_dataframe(dataset):
    """ convert an out file into a pandas DataFrame
        Arguments:
            dataset (str): path to the OUT file
        Returns:
            a pandas DataFrame
    """
    file = open(dataset)
    file_content = file.readlines()
    clean_file_content = []
    for line in file_content:
        line = line.strip() # removes the trailing \n from each line
        clean_file_content.append(line)

    # header parsing
    start_of_header = clean_file_content.index('START-OF-FIELDS')
    end_of_header = clean_file_content.index('END-OF-FIELDS')
    header = clean_file_content[start_of_header+1:end_of_header]

    # data parsing
    start_of_data = clean_file_content.index('START-OF-DATA')
    end_of_data = clean_file_content.index('END-OF-DATA')
    data = clean_file_content[start_of_data+1:end_of_data]
    
    clean_data = []
    for row in data:
        row_data = row.split('|')
        # ignore first three columns, final | has an empty value at the end
        row_data = row_data[3:-1]
        clean_data.append(row_data)

    dataframe = pd.DataFrame(data=clean_data, columns=header)
    return dataframe

In [ ]:
df = out_to_dataframe('data/20191204_best_amer.out')

In [ ]:
df.head()